In [1]:
import sqlite3

DB_PATH = "etymology.db"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()


In [4]:
word = "εἴδομαι"  

# 1. Find all node_ids for this word (could be multiple POS/etymologies)
rows = cur.execute(
    """
    SELECT node_id, lang_code, pos, etymology_number
    FROM entries
    WHERE word = ?
    """,
    (word,)
).fetchall()

if not rows:
    print(f"No entries found for {word!r}")
else:
    print(f"Entries for {word!r}:")
    for node_id, lang, pos, etyno in rows:
        print(f"  node_id={node_id}, lang={lang}, pos={pos}, etymology_number={etyno}")

    print("\nEdges:")
    for node_id, _, _, _ in rows:
        edge_rows = cur.execute(
            """
            SELECT src_id, dst_id, relation_type
            FROM edges
            WHERE src_id = ?
            UNION ALL
            SELECT src_id, dst_id, relation_type
            FROM edges
            WHERE dst_id = ?
            """,
            (node_id, node_id)
        ).fetchall()

        if not edge_rows:
            print(f"  (no edges for {node_id})")
        else:
            print(f"  Edges for {node_id}:")
            for src, dst, rel in edge_rows:
                print(f"    {src} --[{rel}]--> {dst}")

Entries for 'εἴδομαι':
  node_id=grc:εἴδομαι:verb:0, lang=grc, pos=verb, etymology_number=None

Edges:
  Edges for grc:εἴδομαι:verb:0:
    grc:εἴδομαι:verb:0 --[derived]--> grc:εἶδος:noun:0
    grc:εἴδομαι:verb:0 --[derived]--> grc:ἰδέα:noun:0
    grc:εἴδομαι:verb:0 --[derived]--> grc:ἱστορία:noun:0
    grc:εἴδομαι:verb:0 --[derived]--> grc:ἵστωρ:noun:0
